<H1> This is the week 3 assignment. 

<H3> resolving the environment with all necessary imports 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import geocoder as gc 
import folium as fl
from bs4 import BeautifulSoup as bs
import xml
import matplotlib as mlb
import json
import requests

<H4> Importing of all packages done!, now arranging url for data extraction from wikipedia 

In [2]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = bs(url,'lxml')

<H4> Identify the table and extract the data  as mentioned in instructions according to Borough, neighbourhood

In [3]:
table_post = soup.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):   # to iterate through the xml data and creating 3 different lists (columns) for df
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
df_PostCode = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_PostCode.columns = ['Postcode', 'Borough', 'Neighbourhood'] #renaming the columns to caps 
df_PostCode.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<H4> Remove Not Assigned values from dataframe and replace them by the aggregate values

In [4]:
df_PostCode.head(7)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
df_PostCode['Borough'].replace('Not assigned', np.nan, inplace=True)
df_PostCode.dropna(subset=['Borough'], inplace=True)

df_PostCode.head(7)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [6]:
df_PostCodeNeigh = df_PostCode.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_PostCodeNeigh.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_PostCodeNeigh

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [7]:
df_PostCodeNeigh['Neighbourhood'].replace('Not assigned', "Queen's Park", inplace=True) # replacing the 'Not assigned with 'Queen's Park'
df_PostCodeNeigh

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [8]:
df_PostCodeNeigh.shape  # to check the shape of dataframe for merge operation

(103, 3)

<H4> Adding the Geospatial data for Canada as a dataframe, and making sure the columns are named as er requirement for merge operation

In [9]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.columns = ['Postcode', 'Latitude', 'Longitude']

In [10]:
df_geo.shape  # to ckeck the shape of the datafram for merge

(103, 3)

<h4> Concluding the merge operation on both the data frames

In [11]:
df_pos = pd.merge(df_PostCodeNeigh, df_geo, on=['Postcode'], how='inner')

df_tor = df_pos[['Borough', 'Neighbourhood', 'Postcode', 'Latitude', 'Longitude']].copy()

df_tor.head()

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
